# Lista 3

### Exercicio 1

In [88]:
# Imports
import sys

from scipy.optimize import minimize

sys.path.append(f'../FinanceHub')

from calendars import DayCounts
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [89]:
# Functions related to DI
def calc_du(reference_date, expiry_date):
    t = pd.to_datetime(reference_date)
    dc = DayCounts('BUS/252', calendar='anbima')
    T = pd.to_datetime(expiry_date)
    return dc.days(t, T)


def pu_DI(yield_rate, du):
    return 100000 / ((1 + yield_rate) ** (du / 252))


def calc_macaulay(du):
    return du/252


def modified_duration_DI(macaulay, yield_rate):
    return macaulay/(1+yield_rate)


def convexity_DI(yield_rate, macaulay):
    return macaulay*(macaulay+1)/((1+yield_rate)**2)


In [90]:
starting_date = '2021-02-05'

DI_yield_prices = [1.902, 1.98, 2.051, 2.19, 2.36, 2.548,
                   2.742, 2.895, 3.047, 3.221, 3.42, 3.586,
                   3.91, 4.325, 4.71, 4.98, 5.385, 5.49,
                   5.72, 5.82, 5.97, 6.11, 6.221, 6.31,
                   6.457, 6.547, 6.643, 6.69, 6.837, 6.98,
                   7.21, 7.42, 7.603, 7.76, 7.952, 8.091]

DI_dates = [
    '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01',
    '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
    '2022-01-01', '2022-02-01', '2022-04-01',
    '2022-07-01', '2022-10-01', '2023-01-01', '2023-04-01', '2023-07-01',
    '2023-10-01', '2024-01-01', '2024-07-01', '2024-10-01', '2025-01-01',
    '2025-04-01', '2025-07-01', '2025-10-01',
    '2026-01-01', '2026-07-01', '2027-01-01', '2028-04-01', '2029-01-01',
    '2030-01-01', '2031-01-01', '2033-01-01', '2035-01-01', '2036-01-01',
]

DI_df = pd.DataFrame({'price': DI_yield_prices, 'date': DI_dates})
DI_df["du"] = DI_df.apply(
    lambda row: calc_du(starting_date, row["date"]), axis=1, result_type="expand"
)
DI_df["yield"] = DI_df["price"]/100
DI_df["pu"] = pu_DI(DI_df["yield"], DI_df["du"])
DI_df["macaulay"] = calc_macaulay(DI_df["du"])
DI_df["modified"] = modified_duration_DI(DI_df["macaulay"], DI_df["yield"])
DI_df["convexity"] = convexity_DI(DI_df["macaulay"], DI_df["yield"])

In [91]:
# Necessary functions to analyze the portfolio

# Functions related to LTN
# Calculate the yield of a LNT given parameters
def rate_ltn(principal, pu, du):
    rate = (principal/pu)**(252/du) - 1
    return rate

# Macaulay duration of a LTN
def macaulay_ltn(du):
    return du/252

def convexity_LTN(du):
    return (du/252)*(du/252+1)

# Modified duration of LTN
def modified_duration(macaulay, rate):
    return macaulay/(1+rate)

In [92]:
# Functions related to NTNF
# Payout dates from starting date to expiration of a NTNF
def payout_dates_ntnf(reference_date, expiry_date):
    dc = DayCounts('BUS/252', calendar='anbima')

    t = dc.following(pd.to_datetime(reference_date))
    T = dc.following(pd.to_datetime(expiry_date))

    d_years = 2 * (T.year - t.year) - 1
    # Check what's the first payout day
    first_day = pd.to_datetime(f"{t.year}-01-01")
    second_day = pd.to_datetime(f"{t.year}-07-01")
    if t != first_day:
        if t > second_day:
            d_years -= 1
            starting_day = pd.to_datetime(f"{t.year+1}-01-01")
        else:
            starting_day = second_day
    else:
        starting_day = first_day

    output = [dc.following(starting_day)]
    # Iterate over the years and append business day for payouts
    for i in range(0, d_years-1):
        if output[i].month == 1:
            output.append(
                dc.following(pd.to_datetime(f"{output[i].year}-07-01"))
            )
        else:
            output.append(
                dc.following(pd.to_datetime(f"{output[i].year+1}-01-01"))
            )
    # Add last payout date, expiry
    output.append(dc.following(pd.to_datetime(expiry_date)))

    return output

# Calculate current price of a NTF
def pu_ntnf(reference_date, expiry_date, yield_rate, coupon=0.1, face_value=1000):
    dc = DayCounts('BUS/252', calendar='anbima')
    # Calculate the payout dates
    payout_days = payout_dates_ntnf(reference_date, expiry_date)

    # First coupon value from reference date
    t = dc.following(pd.to_datetime(reference_date))
    du = dc.days(t, payout_days[0])
    # Constant values: Coupon value and yield rate
    c = (1 + coupon)**0.5 - 1
    y = (1 + yield_rate)
    _pu = c/(y**(du/252))
    for i in range(1, len(payout_days)):
        # Calculate business days between each payout
        du += dc.days(payout_days[i-1], payout_days[i])
        # Sums up coupon values
        _pu += c/(y**(du/252))

    # Last payout of face value
    _pu += 1/(y**(du/252))

    return face_value*_pu

# Calculate the yield of a NTNF
def yield_ntnf(reference_date, expiry_date, price, coupon=0.1, face_value=1000, estimate=0.05):
    from scipy import optimize
    get_yield = lambda yield_rate: pu_ntnf(reference_date, expiry_date, yield_rate) - price
    # Use the function above and an estimate of the yield rate to optimize
    # the function until we find a good enough value for the yield so that:
    # calc_pu(estimate_yield) = price
    return optimize.newton(get_yield, estimate)

# Calculates present value, modified duration and convexity of a NTNF
def ntnf(reference_date, expiry_date, yield_rate, coupon=0.1, face_value=1000):
    dc = DayCounts('BUS/252', calendar='anbima')
    # Calculate the payout dates
    payout_days = payout_dates_ntnf(reference_date, expiry_date)

    yield_price = 0
    macaulay = 0
    convexity = 0

    # First coupon value from reference date
    t = dc.following(pd.to_datetime(reference_date))
    du = dc.days(t, payout_days[0])
    # Constant values: Coupon value and yield rate
    c = face_value*((1 + coupon) ** 0.5 - 1)
    y = (1 + yield_rate)
    for i in range(0, len(payout_days)):
        # Calculate business days between each payout
        du = dc.days(t, payout_days[i])
        # Sums up coupon values
        cf = c / (y ** (du / 252))
        yield_price += cf
        macaulay += cf*(du/252)
        convexity += cf*(du/252)*(du/252+1)

    # Last payout of face value
    cf = face_value / (y ** (du / 252))
    yield_price += cf

    macaulay += cf * (du / 252)
    macaulay /= yield_price
    md = modified_duration(macaulay, yield_rate)

    convexity += cf * (du / 252) * (du / 252 + 1)
    convexity /= (yield_price*(1+yield_rate)**2)

    return [yield_price, md, convexity]

In [93]:
# Gets expiration date from description
def du_from_desc(reference_date, description):
    # BLTN 0 04/01/21
    t = pd.to_datetime(reference_date)
    date = description.split(" ")[-1]
    T = pd.to_datetime(date)
    dc = DayCounts('BUS/252', calendar='anbima')

    return dc.days(t, T)

# Return the current price, modified duration and convexity of a bond, Shift in bps
def analyze(row, shift=0):
    if "LTN" in row["Description"]:
        macaulay = macaulay_ltn(row["du"])
        yield_rate = rate_ltn(1000, row["Vendor Price"], row["du"])
        yield_rate += shift/10000
        md = modified_duration(macaulay, yield_rate)
        convexity = convexity_LTN(row["du"])
        yield_price = 1000.0/(1+yield_rate)**(row["du"]/252)

        return [yield_price, md, convexity]

    elif "NTNF" in row["Description"]:
        expiry_date = row["Description"].split(" ")[-1]
        yield_rate = yield_ntnf(starting_date, expiry_date, row["Vendor Price"])
        yield_rate += shift/10000
        analyzed = ntnf(starting_date, expiry_date, yield_rate)
        return analyzed

# Delta of a shift in bps for a bond
def delta(price, md, c, dbps):
    d = price * (md*(-1/dbps) + (c/2)*((1/dbps)**2))
    return d

In [94]:
# Analyze the portfolio
portfolio = pd.read_excel("BZRFIRFM Index as of Feb 05 20211.xlsx")
portfolio["Weight"] = portfolio["Weight (%)"]/100
portfolio["du"] = portfolio.apply(lambda row: du_from_desc(starting_date, row["Description"]), axis=1)
portfolio[["Yield Price", "Modified Duration", "Convexity"]] = portfolio.apply(analyze, axis=1, result_type="expand")

portfolio_price = (portfolio["Market Value"]*portfolio["Weight"]).sum()
portfolio_md = (portfolio["Modified Duration"]*portfolio["Weight"]).sum()
portfolio_convexity = (portfolio["Convexity"]*portfolio["Weight"]).sum()

print("Valor do portfolio: " + str(portfolio_price))
print("Modified Duration: " + str(portfolio_md))
print("Convexity: " + str(portfolio_convexity))

Valor do portfolio: 169296826.2049501
Modified Duration: 1.7554103311691713
Convexity: 7.7668715211463795


In [95]:
dbps = 100
delta_portfolio = delta(portfolio_price, portfolio_md, portfolio_convexity, dbps)

print("Delta de 100 bps do porfolio: " + str(delta_portfolio))

Delta de 100 bps do porfolio: -2906108.642639626


In [96]:
# Contracts chosen as the expiration date falls "in the middle" of the DI curve
hedge_1 = DI_df[DI_df["date"] == "2022-07-01"]
hedge_2 = DI_df[DI_df["date"] == "2024-07-01"]
print("PU do primeiro DI: " + str(hedge_1["pu"].item()))
print("PU do segundo DI: " + str(hedge_2["pu"].item()))

delta_hedge_1 = delta(hedge_1["pu"].item(), hedge_1["modified"].item(), hedge_1["convexity"].item(), dbps)
delta_hedge_2 = delta(hedge_2["pu"].item(), hedge_2["modified"].item(), hedge_2["convexity"].item(), dbps)
print("Delta de 100 bps do primeiro DI: " + str(delta_hedge_1))
print("Delta de 100 bps do segundo DI: " + str(delta_hedge_2))

PU do primeiro DI: 94273.05802412987
PU do segundo DI: 82216.24363364592
Delta de 100 bps do primeiro DI: -1258.615165222524
Delta de 100 bps do segundo DI: -2620.0010493960613


In [97]:
# Fixing number of second hedge contracts
N2 = 600

N1 = (delta_portfolio - N2*delta_hedge_2)/delta_hedge_1
print("Numero de contratos primeiro DI: " + str(N1))
print("Numero de contratos segundo DI: " + str(N2))

hedge_price = N1*hedge_1["pu"].item() + N2*hedge_2["pu"].item()


Numero de contratos primeiro DI: 1059.980882056286
Numero de contratos segundo DI: 600


### Exercise 2

Os titulos foram escolhidos de forma que as datas de expiração ficassem próximas as datas de expiração do portfolio (ele tem bastante vencimentos em 2022, então um dos contratos DI escolhidos foi o de vencimento em 2022). O outro contrato foi escolhido para ter um vencimento maior, mas não tão longe de forma que não se possa rebalancear devido à liquidez, por isso escolheu-se 2024

### Exercise 3

In [98]:
# Calculate apply the yield shift to the market value
def pct_diff(row):
    pct = row["Yield Price"]/row["Vendor Price"]
    market_value = pct*row["Market Value"]
    return market_value
# Apply a shift to the yield curve
shift_bps = 150
portfolio_shift = portfolio
portfolio_shift[["Yield Price", "Modified Duration", "Convexity"]] = portfolio.apply(
    lambda row: analyze(row, shift_bps), axis=1,result_type="expand"
)
portfolio_shift["Market Value"] = portfolio_shift.apply(pct_diff, axis=1, result_type="expand")
portfolio_shift_price = (portfolio_shift["Market Value"]*portfolio_shift["Weight"]).sum()
print("Valor do portfolio depois do shift de 150 bps: " + str(portfolio_shift_price))

Valor do portfolio depois do shift de 150 bps: 165889259.1169103


In [99]:
# Shift the hedge
shift_hedge_1 = hedge_1.copy(deep=True)
shift_hedge_2 = hedge_2.copy(deep=True)

shift_hedge_1["yield"] += shift_hedge_1["yield"] + shift_bps/10000
shift_hedge_2["yield"] += shift_hedge_2["yield"] + shift_bps/10000
shift_hedge_1["pu"] = pu_DI(shift_hedge_1["yield"], shift_hedge_1["du"])
shift_hedge_2["pu"] = pu_DI(shift_hedge_2["yield"], shift_hedge_2["du"])

print("Valor do primeiro contrato DI depois do shift de 150 bps: " + str(shift_hedge_1["pu"].item()))
print("Valor do segundo contrato DI depois do shift de 150 bps: " + str(shift_hedge_2["pu"].item()))

Valor do primeiro contrato DI depois do shift de 150 bps: 87402.04359194613
Valor do segundo contrato DI depois do shift de 150 bps: 65321.380069116654


In [100]:
delta_shift_hedge = -N1*(shift_hedge_1["pu"].item()-hedge_1["pu"].item())\
                    -N2*(shift_hedge_2["pu"].item()-hedge_2["pu"].item())

pnl_hedgeless = portfolio_shift_price - portfolio_price
pnl_hedgeless_pct = pnl_hedgeless/portfolio_price * 100
pnl_hedge = pnl_hedgeless + delta_shift_hedge
pnl_hedge_pct = pnl_hedge/portfolio_price * 100

print("PNL without Hedge: " + str(pnl_hedgeless))
print("PNL without Hedge %: " + str(pnl_hedgeless_pct))
print("PNL with Hedge: " + str(pnl_hedge))
print("PNL with Hedge %: " + str(pnl_hedge_pct))

PNL without Hedge: -3407567.0880397856
PNL without Hedge %: -2.012776709655855
PNL with Hedge: 14012494.989125371
PNL with Hedge %: 8.276879905688189
